In [61]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.layers import Dense
from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

from keras.utils import np_utils
from keras.utils.vis_utils import plot_model

from skimage.transform import resize 
import numpy as np
import pandas as pd
import cv2


In [7]:
model_res = ResNet50(include_top = False, weights = 'imagenet', input_tensor = None, input_shape = (197,197,3), pooling = 'avg' , classes = 7)

In [8]:


x = model_res.output
#x = Dense(2048, activation ='relu')(x)
predictions = Dense(7, activation ='softmax')(x)

model = Model(inputs = model_res.input, outputs = predictions)

for layer in model_res.layers:
    layer.trainable = False


In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 203, 203, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [49]:
class_label = ['angry', 'disgust', 'fear','happy','sad','surprise','neutral']
n_class = len(class_label)

path = '/python/DDSA/fer/fer2013/'
file_name = 'fer2013_2.csv'

n_data = 1000

def load_data():
    print("Start load_data")
    fer = pd.read_csv(path + file_name)
    fer_train = fer[fer.Usage == 'Training']
    fer_test = fer[fer.Usage.str.contains('Test', na=False)]

    x_train = np.array([list(map(int, x.split())) for x in fer_train['pixels'].values])
    y_train = np.array(fer_train['emotion'].values)

    x_test = np.array([list(map(int, x.split())) for x in fer_test['pixels'].values])
    y_test = np.array(fer_test['emotion'].values)
    
    # to deal with the memory issue.. 
    x_train = x_train[0:n_data]
    x_test = x_test[0:n_data]
    y_train = y_train[0:n_data]
    y_test = y_test[0:n_data]
    
    x_train = normalize_x(x_train)
    x_test = normalize_x(x_test)
    
    y_train = np_utils.to_categorical(y_train, n_class)
    y_test = np_utils.to_categorical(y_test, n_class)

    return x_train, x_test, y_train, y_test

def normalize_x(data):
    faces = []
    
    for face in data:
        face = face.reshape(48, 48) / 255.0
        face = cv2.resize(face, (197, 197))
        
        faces.append(face)
        #np.concatenate(faces,face)
    faces = np.asarray(faces)
    #del(data)
    
    #faces = np.expand_dims(faces, -1)
    faces = np.stack((faces,)*3, -1 )  # to make RGB channel
    return faces


In [50]:
x_train, x_test, y_train, y_test = load_data()
np.shape(x_train)

Start load_data


(26197, 48, 48, 3)

In [27]:
epochs = 10

model.compile(loss = categorical_crossentropy,
                      optimizer=Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-7),
                      metrics=['accuracy'])

hist = model.fit(x_train, y_train, 
                              validation_split = 0.2, 
                              shuffle = True, 
                              batch_size = 16, epochs = epochs, verbose = 1, 
                               )

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 312s 390ms/step - loss: 1.8529 - acc: 0.2675 - val_loss: 1.9181 - val_acc: 0.1300
Epoch 2/10
800/800 [==============================] - 308s 385ms/step - loss: 1.5365 - acc: 0.4175 - val_loss: 2.0232 - val_acc: 0.1950
Epoch 3/10
800/800 [==============================] - 308s 385ms/step - loss: 1.4018 - acc: 0.4788 - val_loss: 2.0073 - val_acc: 0.2000
Epoch 4/10
800/800 [==============================] - 321s 402ms/step - loss: 1.2897 - acc: 0.5162 - val_loss: 1.9892 - val_acc: 0.1900
Epoch 5/10
800/800 [==============================] - 325s 407ms/step - loss: 1.1364 - acc: 0.6000 - val_loss: 2.1156 - val_acc: 0.1900
Epoch 6/10
800/800 [==============================] - 324s 405ms/step - loss: 1.0760 - acc: 0.6225 - val_loss: 2.1743 - val_acc: 0.1900
Epoch 7/10
800/800 [==============================] - 314s 393ms/step - loss: 1.0092 - acc: 0.6637 - val_loss: 2.2276 - val_acc: 0.1900
Ep

In [28]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 197, 197, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 203, 203, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 99, 99, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 99, 99, 64)   256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [33]:
import matplotlib.pyplot as plt
import glob
import numpy as np
import pandas as pd
import cv2

from sklearn.metrics import confusion_matrix #classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import model_from_json
from keras.models import load_model

from keras.callbacks import EarlyStopping, TensorBoard

import matplotlib.pyplot as plt
#%matplotlib inline  # for jupyter notebook enviornment. 

import itertools  # for confusion matrix plot

from PIL import Image as pil_image

def plot_hist(hist):
    plt.figure(0)
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')

    acc_ax.plot(hist.history['acc'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='lower left')

    plt.show()
    plt.savefig('loss_accuracy_plot')
    plt.close()

# Make and plot confusion matrix. To see the detailed imformation about TP, TN of each classes.    
def make_confusion_matrix(model, x, y, normalize = True):
    predicted = model.predict(x)

    pred_list = []; actual_list = []
    for i in predicted:
        pred_list.append(np.argmax(i))
    for i in y:
        actual_list.append(np.argmax(i))

    confusion_result = confusion_matrix(actual_list, pred_list)
    plot_confusion_matrix(confusion_result, classes = class_label, normalize = normalize, title = 'Confusion_matrix')
    return confusion_result

def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix',
                          cmap = plt.cm.Blues):  
    if normalize:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        print("normalized")
    else:
        print('without normalization')

    print(cm)
    plt.figure(1)
    plt.imshow(cm, interpolation='nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment = "center",
                 color = "white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('Confusion_matrix')
    plt.close()

# preprocess for gradCAM
def preprocess(img_path):
    
    img = pil_image.open(img_path)
    img = img.resize((img_size, img_size))
    img_arr = np.asarray(img) / 255.
    img_tensor = np.expand_dims(img_arr, 0)
    img_tensor = np.expand_dims(img_tensor, 3)    
    
    return img_arr, img_tensor
#
#
def grad_cam(model, img_path, class_idx, layer_idx):

    img_arr, img_tensor = preprocess(img_path)

    y_c = model.layers[-1].output.op.inputs[0][0, class_idx]  # final layer (before softmax)
    layer_output = model.layers[layer_idx].output  # pick specific layer output (caution: conv layer only)
    
    grad = K.gradients(y_c, layer_output)[0]  # calculate gradient of y_c w.r.t. A_k from the conv layer output
    gradient_fn = K.function([model.input], [layer_output, grad, model.layers[-1].output])
    
    conv_output, grad_val, predictions = gradient_fn([img_tensor])
    conv_output, grad_val = conv_output[0], grad_val[0]
    
    weights = np.mean(grad_val, axis=(0, 1))
    cam = np.dot(conv_output, weights)
    cam = cv2.resize(cam, (img_size, img_size))
    
    # relu 
    cam = np.maximum(cam, 0)    
    cam = cam / cam.max()
    
    return img_arr, cam, predictions
    
        
def plot_grad_cam(model, img, pred_class=1, layer_idx = -5):
        
    img = np.expand_dims(img, 0)
    #pred_class = np.argmax(model.predict(img))
   
    plt.figure(2)
    fig, axes = plt.subplots(1, 2, figsize=(30, 24))
    img, cam, predictions = grad_cam(model.model, img, pred_class, layer_idx) #in case of model class, model.model
        
    pred_values = np.squeeze(predictions, 0)
    top1 = np.argmax(pred_values)
    top1_value = np.round(float(pred_values[top1]*100), decimals = 4)
    top4 = np.argpartition(pred_values, -4)[-4:]  #top 4
    
    axes[0, 0].set_title("Pred:{}{}%\n True:{}\n{}".format(class_label[top1], top1_value, class_label[pred_class], top4 ), fontsize=10)
    axes[0, 0].imshow(img,cmap = 'gray')
    axes[0, 1].imshow(img,cmap = 'gray')
    axes[0, 1].imshow(cam, cmap = 'jet', alpha = 0.5)


In [34]:
make_confusion_matrix(model, x_test, y_test)

normalized
[[0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]]


array([[  0,   0,   0,   0,   0,   0, 139],
       [  0,   0,   0,   0,   0,   0,  19],
       [  0,   0,   0,   0,   0,   0, 143],
       [  0,   0,   0,   0,   0,   0, 248],
       [  0,   0,   0,   0,   0,   0, 154],
       [  0,   0,   0,   0,   0,   0, 125],
       [  0,   0,   0,   0,   0,   0, 172]], dtype=int64)

In [47]:
## batch test
import keras as k
import keras.preprocessing.image

In [51]:
datagen = ImageDataGenerator(rescale = 4, validation_split = 0.2)
datagen.fit(x_train)


C:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [60]:
np.shape(image_resized)

(197, 197, 3)

Epoch 1/2


ValueError: Error when checking input: expected input_1 to have shape (197, 197, 3) but got array with shape (48, 48, 3)

In [37]:
test=np.array([1,2,3,4,5]

In [44]:
np.delete(test, test[test==2],0)

array([1, 2, 4, 5])

In [ ]:
pred_class = np.argmax(y_test[0])
plot_grad_cam(model, x_test[0], pred_class, layer_idx = -3)